<a href="https://colab.research.google.com/github/sergiorolnic/datascience/blob/main/datascience_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Box Office Prediction
**Programmazione di Applicazioni Data Intensive**

Ingegneria e Scienze Informatiche 2021

Sergiu Gabriel Rolnic

Il seguente progetto riguarda l'analisi  dei dati raccolti dal sito [The Movie Database](https://www.themoviedb.org/). Lo scopo finale sarà quello di prevedere gli incassi di un film avendo a disposizione solo dati esistenti prima della effettiva uscita nelle sale.


# **Analisi dei Dati**

Importazione del dataset e delle librerie utili per la sua analisi

In [ ]:
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
if not os.path.exists("train.csv"):
    from urllib.request import urlretrieve
    urlretrieve("https://raw.githubusercontent.com/sergiorolnic/datascience/main/train.csv", "train.csv")

data = pd.read_csv("train.csv")    

In [ ]:
data.shape

In [ ]:
data.head(1)

In [ ]:
data.tail(1)

Notiamo la presenza di feature particolari: dizionari con una o piu' entrate, valori numerici su scale diverse, stringhe di varia natura. La challenge principale sarà quella di riuscire ad estrarre delle informazione utili da piu' variabili possibili.   

La colonna "id" può essere usata come indice del nostro dataframe

In [ ]:
data = pd.read_csv("train.csv", index_col=0)  


In [ ]:
data.head(1)

## Gestione Features



*   *belongs_to_collection*: appartenenza ad una serie cinematografica
*   *budget*: costo di produzione
*   *genres*: generi
*   *homepage*: link al sito web


*   *imdb_id*: id imbd

*   *original_language*: lingua originale

*   *original_title*: titolo originale

*   *overview*: plot
*   *popularity*: popolarità
*   *poster_path*: path della locandina ufficiale
*   *production_companies*: compagnia di produzione
*   *production_countries*: paese di produzione
*   *release_date*: data di uscita
*   *runtime*: durata
*   *spoken_languages*: lingua del film
*   *status*: stato del rilascio
*   *tagline*: slogan film
*   *title*: titolo
*   *Keywords*: parole chiavi
*   *cast*: cast
*   *crew*: trop
*   *revenue*: incasso del film --> variabile da predire

In [ ]:
data.info(memory_usage="deep")

In [ ]:
data.describe()

La variabile budget e la variabile target revenue presentano dei valori molto alti e diversificati che potrebbero causare dei problemi in fase di addestramento. 

Verifichiamo la presenza di valori null


In [ ]:
data.isnull().sum()

 Notiamo che **Belongs_to_collection** e **Homepage** presentano una preponderanza di valori null. Si decide perciò di binarizzarle attraverso il quesito "Is Present?" in quanto sono features che potrebbero risultare particolarmente utili durante l'addestramento.

In [ ]:
data['belongs_to_a_collection'] = data['belongs_to_collection'].apply(lambda x: 0 if pd.isna(x) else 1)
data = data.drop(columns='belongs_to_collection')

data['homepage_is_present'] = data['homepage'].apply(lambda x: 0 if pd.isna(x) else 1)
data = data.drop(columns='homepage')



**Budget**



Osserviamo la distribuzione del budget e della sua funzione logaritmica in modo da vedere se sia possibile uniformare meglio i valori elevati presenti.

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1).hist(data.budget,50)
plt.subplot(1, 2, 2).hist(np.log1p(data.budget),50);



In [ ]:
data.budget[data.budget>1000000].count()

La quasi totalità dei valori non nulli sono superiori al milione, perciò possiamo togliere le righe con valori inferiori.

In [ ]:
data = data[data.budget > 1000000]

Si usa la funzione "log(1+x)" per trasformare la variabile, uniformando i dati e diminuendo il peso dei grandi blockbuster. 

In [ ]:
data.budget = np.log1p(data.budget)

In [ ]:
data.budget.plot.hist(bins=50);


Si procede allo stesso modo con l'analisi della variabile target **revenue**

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1).hist(data.revenue,50)
plt.subplot(1, 2, 2).hist(np.log(data.revenue),50);

In [ ]:
data.revenue = np.log1p(data.revenue)

**Budget** e **revenue** presentano entrambe una distribuzione molto simile, perciò si decide di verificare un'eventuale correlazione tra le due. 

In [ ]:
plt.scatter(data.budget, data.revenue);


In [ ]:
np.mean((data.budget-data.budget.mean()) * (data.revenue-data.revenue.mean())) / (data.budget.std() * data.revenue.std())

Usando la correlazione di Pearson si ottiene una buona correlazione tra le due variabili

**Genres**

Il genere è la prima feature di tipo dizionario. I valori distinti sono limitati, perciò si procede con la binarizzazione di tutti gli elementi

In [ ]:
data["genres"] = (data['genres'].apply(lambda x: [i['name'] for i in eval(x)] if str(x) != 'nan' else []).values)
list_of_genres = {i for x in data.genres for i in x}
for genre in list_of_genres:
  data["genre_" + genre] = data['genres'].apply(lambda x: 1 if genre in x else 0)
data.drop(columns='genres',inplace=True)



Per quanto riguara le features **imdb_id, 'original_title', 'status', 'poster_path'** e **title** di eliminarle in quanto poco significative

In [ ]:
data.drop(columns=['imdb_id','original_title','status','poster_path', 'title'], inplace=True)


**Original_language**



In [ ]:
data.original_language.value_counts(normalize=True).head(5).plot.pie();


Per quanto a primo impatto potesse risultare un dato interessante, la maggior parte dei film sono in inglese, quindi risulterebbero inutili in fase di addestramento. Lo stesso ragionamento vale per **spoken_language**
Un discorso diverso invece va fatto con **popularity**. Non avendo certezza del fatto che il rating di popolarità sia antecedente alla fuoriuscita dei film, si decide cancellare la colonna assieme alle altre.

In [ ]:
#data.drop(columns= ['original_language','spoken_languages', 'popularity'], inplace=True)
data.drop(columns= ['original_language','spoken_languages'], inplace=True)


**Keywords**

In [ ]:
data.Keywords.head()

**tagline** e **overview** svolgono la stessa funzione di **Keywords**, e cioè l'estrazione di parole chiavi da da associare a ciascun film. Si procede ad eliminarle.

In [ ]:
data.drop(columns=['overview', 'tagline'], inplace=True)



Si definisce una funzione per l'estrazione dei nomi dai dizionari, la selezione dei 100 nomi piu' diffusi e la loro binarizzazione.

In [ ]:
def cut_and_binariezed(feature):
  
  data[feature] = (data[feature].apply(lambda x: [i['name'] for i in eval(x)] if str(x) != 'nan' else []).values)
  all_values = pd.DataFrame([i for x in data[feature] for i in x])
  
  split = all_values.value_counts()[:10]
  for keys in split.index:
      data[feature+" (" + keys[0]+ ")"] = data[feature].apply(lambda x: 1 if keys[0] in x else 0)
  data.drop(columns=feature,inplace=True) 



In [ ]:
cut_and_binariezed("Keywords")

Si procede allo stesso modo con **cast**,**crew**,  **production_companies** e **production_countries**

In [ ]:
for feature in ["cast","crew", "production_companies","production_countries"]:
  cut_and_binariezed(feature)

**Release Date**

Vengono create 3 feature rappresentanti giorno,mese, anno

In [ ]:
data[['release_month', 'release_day', 'release_year']] = data['release_date'].str.split('/', expand=True).astype(int)

In [ ]:
plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1).bar(data.release_month.value_counts().index,data.release_month.value_counts().values)
plt.subplot(1, 3, 2).bar(data.release_day.value_counts().index,data.release_day.value_counts().values)
plt.subplot(1, 3, 3).bar(data.release_year.value_counts().index,data.release_year.value_counts().values)


Alcuni valori della feature **release_year** presentano solo le ultime cifre, perciò vanno uniformate 

In [ ]:
data["release_year"]=data['release_year'].map(lambda x: 2000 + x if x < 20 else (x+1900 if x<100 else x))
plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1).bar(data.release_month.value_counts().index,data.release_month.value_counts().values)
plt.subplot(1, 3, 2).bar(data.release_day.value_counts().index,data.release_day.value_counts().values)
plt.subplot(1, 3, 3).bar(data.release_year.value_counts().index,data.release_year.value_counts().values)

In [ ]:
data.drop(columns='release_date', inplace=True)

In [ ]:
data.head(1)

**Runtime**

In [ ]:
data.plot.scatter("runtime", "revenue");

In [ ]:
data

# Addestramento


Prima di procedere all'addestramento controlliamo la presenza di valori null e e nel caso rimuoviamo le righe corrispondenti

In [ ]:
data.isna().sum()

In [ ]:
data.dropna(inplace=True)


In [ ]:
data.shape

In [ ]:
Y_data = data.revenue
X_data = data.drop(columns='revenue')


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate

from sklearn.metrics import r2_score, mean_squared_error , mean_absolute_error

In [ ]:
import sklearn.preprocessing

In [ ]:

def relative_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_eval(X, y, model):
    preds = model.predict(X)
    
    print("   Mean squared error: ",mean_squared_error( y, preds))
    print("    Relative error: ",relative_error( y, preds))
    print("       Mean absolute_error: ",mean_absolute_error(y, preds))
    print("R-squared coefficient: ",r2_score(y, preds))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = \
    train_test_split(X_data, Y_data, test_size=1/3, random_state=42)

In [ ]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("lrn",  LinearRegression())
])
linear_model = model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [ ]:
model = Pipeline([
    ("scale",  StandardScaler(with_mean=False)),
    ("linreg", Lasso())
])
grid = {
    "linreg__alpha": [0.001,0.01, 0.1, 1]
    }
gs = GridSearchCV(model, param_grid=grid, scoring="neg_mean_squared_error", cv=5)
result = gs.fit(X_data, Y_data)
lasso_model = result.best_estimator_
sel = ["mean_test_score","params"]
pd.DataFrame(gs.cv_results_).sort_values("mean_test_score", ascending=False)[sel]

In [ ]:
model = Pipeline([
    ("scale",  StandardScaler(with_mean=False)),
    ("linreg", Lasso(alpha=0.01))
])
model.fit(X_train, y_train);
print_eval(X_val, y_val, model)

In [ ]:
lasso = pd.Series(model.named_steps["linreg"].coef_, X_data.columns)
lasso.sort_values(inplace=True)
lasso.head(5)

In [ ]:
lasso.tail(5)

In [ ]:
from sklearn.linear_model import ElasticNet, Ridge

In [ ]:
model = Pipeline([
    ("scale",  StandardScaler(with_mean=False)),
    ("reg", ElasticNet())
])
grid = {
    "reg__alpha": [0.01, 0.1, 1, 10],
    "reg__l1_ratio": [0.1, 0.5, 1]
    }
gs = GridSearchCV(model, param_grid=grid, scoring="neg_mean_squared_error", cv=5)
gs.fit(X_data, Y_data)
sel = ["mean_test_score","param_reg__alpha",	"param_reg__l1_ratio"]
pd.DataFrame(gs.cv_results_).sort_values("mean_test_score", ascending=False)[sel]

Grazie alla regressione elastic net che combina insieme le regolarizzazioni L2 e L1 usate in ridge e lasso notiamo che comunque non si riesce a migliore i risultati precedentemente acquisiti

In [ ]:
from sklearn.kernel_ridge import KernelRidge


In [ ]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=0.1, kernel="poly", degree=2))
])
model.fit(X_train, y_train);
print_eval(X_val, y_val, model)

In [ ]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=0.1, kernel="rbf", gamma=0.001))
])
ridge_model = model.fit(X_train, y_train);
print_eval(X_val, y_val, model)

Passiamo alle reti neurali per vedere se ci sono eventuali miglioramenti

In [ ]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(
    hidden_layer_sizes=[10],
    activation="relu",
    solver="lbfgs",
    random_state=12345,
    max_iter=10000
)
model.fit(X_train, y_train);
print_eval(X_val, y_val, model)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(128, activation="relu", input_dim=X_train.shape[1]),
    
    Dense(32, activation="relu"),
    Dense(1)
])
model.compile(optimizer="adam", loss="mean_squared_error")


In [ ]:
history = model.fit(X_train, y_train, batch_size=100, epochs=5)


In [ ]:
model.evaluate(X_val, y_val)

In [ ]:
plt.plot(history.history["loss"], "ro-")
plt.xlabel("Epoche")
plt.ylabel("MSE");

In [ ]:
from tensorflow.keras.regularizers import l2
model = Sequential([
    Dense(128, activation="relu", kernel_regularizer=l2(0.001), input_dim=X_train.shape[1]),
    Dense(1)
])
model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
model.fit(X_train, y_train, batch_size=100, epochs=10)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=2, random_state=42)
model.fit(X_train, y_train);
print_eval(X_val, y_val, model)

In [ ]:
model = DecisionTreeRegressor(min_samples_split=0.1, random_state=42)
model.fit(X_train, y_train);
print_eval(X_val, y_val, model)

In [ ]:
model = DecisionTreeRegressor(random_state=42)
grid = {
    "max_depth": [3, 5, 10, 100],
    "min_samples_split": [10,2, 0.02, 0.05, 0.1]
}
kf = KFold(3, shuffle=True, random_state=42)
gs = GridSearchCV(model, grid, cv=kf)
result = gs.fit(X_train, y_train)
best_model = result.best_estimator_
print_eval(X_val, y_val, best_model)

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures
std_xgb = Pipeline([
   
    ('std', StandardScaler()),
    
    ('xgb', xgb.XGBRegressor(objective ='reg:squarederror'))
])

parameters = {
    'xgb__eta': [ 0.01],
   
    'xgb__max_depth': [6,10],

}

xgb_gs = GridSearchCV(std_xgb, parameters, cv=3, n_jobs=-1, return_train_score=True, scoring='neg_mean_squared_error')
result = xgb_gs.fit(X_train, y_train)
best_model = result.best_estimator_
print_eval(X_val, y_val, best_model)


# Valutazione modelli

Usiamo la **Root Meen Square Error** -- sqrt( mse( y, y_pred) -- per confrontare i modelli migliori tra loro, in sintonia con il criterio di valutazione utilizzato su Kaggle.

In [ ]:
from scipy import stats
from math import sqrt

In [ ]:

def root_mse(mse_model):
  return sqrt(mse_model)



In [ ]:
def confidence_error(r2):
  n = X_val.shape[0]
  f = X_val.shape[1]
  t_value = stats.t.ppf(1-0.025, n)
  dev = (4*r2*((1-r2)**2)*((n-f-1)**2))/(((n**2)-1)*(n+3))
  sqrt_dev = sqrt(dev)
  min = r2- (t_value*sqrt_dev)
  max = r2+ (t_value*sqrt_dev)
  return min,max



In [ ]:
from prettytable import PrettyTable

Aggiungiamo la Root Meen Square Error per confrontare i modelli migliori tra loro, in sintonia con il criterio di valutazione  utilizzato dalla suddetta competizione su Kaggle.

In [ ]:
x = PrettyTable()
y = PrettyTable()

x.field_names = ["Errore","Regressione lineare","Regressione lasso", "kernel ridge rbf", 'MLPRegressor']


x.add_row(['RMSE',root_mse(2.8846),root_mse(2.848063),root_mse(2.7447),root_mse(2.728464) ])
x.add_row(['MAE',1.1260207,1.1073617,1.092814,1.07795 ])
x.add_row(['RE',0.086118 , 0.085253, 0.0838466 ,0.08655 ])

min_linear,max_linear = confidence_error(0.44675)
min_lasso,max_lasso = confidence_error(0.45195)
min_rbf,max_rbf = confidence_error(0.47359)
min_MLPRegressor,max_MLPRegressor = confidence_error(0.47670)

y.field_names = ["R^2 Confidence","Min","Max"]

y.add_row(['Regressione lineare',min_linear,max_linear ])
y.add_row(['Regressione lasso',min_lasso, max_lasso ])
y.add_row(['kernel ridge rbf',min_rbf, max_rbf ])
y.add_row(['MLPRegressor',min_MLPRegressor, max_MLPRegressor ])


print(x)
print(y)

In [ ]:
def model_compare(mse_1, mse_2):
    d = np.abs(mse_1 - mse_2)
    variance = ((mse_1 * (1 - mse_1)) / len(X_val)) + ((mse_2 * (1 - mse_2)) / len(X_val))
    d_min = d - (1.96 * sqrt(variance))
    d_max = d + (1.96 * sqrt(variance))
    return d_min, d_max

**Regressione Lineare vs Regressione Lasso**

In [ ]:
print('Interval {}'.format(np.round(model_compare(0.086118 , 0.085253 ), 4)))

**RBF vs MLPRegressor**

In [ ]:
print('Interval {}'.format(np.round(model_compare(0.0838466, 0.08655), 4)))

In [ ]:
from sklearn.dummy import DummyRegressor


In [ ]:
dummy_regr = DummyRegressor(strategy="median")
dummy_regr.fit(X_train, y_train)
print_eval(X_val, y_val, dummy_regr)

**DummyRegressor vs RBG**

In [ ]:
t_value = stats.t.ppf(1-0.005, X_train.shape[0])
t_value

In [ ]:
def dummy_compare(r2_rbf, r2_dummy):
    t_value = stats.t.ppf(1-0.005, X_train.shape[0])
    d = np.abs(r2_rbf - ms)
    variance = ((r2_rbf * (1 - r2_rbf)) / len(X_val)) + ((r2_dummy * (1 - r2_dummy)) / len(X_val))
    d_min = d - (t_value * sqrt(variance))
    d_max = d + (t_value * sqrt(variance))
    return d_min, d_max

In [ ]:
print('Interval {}'.format(np.round(dummy_compare(0.47359, -0.02382), 4)))

La differenza 